In [1]:
import re,requests
from tqdm import tqdm
import pandas as pd
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

from os import listdir
from os.path import isfile, join


In [2]:
words = ['мм', 'рт', 'ст', 'г', 'м', 'мл', 'кг', 'мг', 'см', 'ч', 'е', 'х', 'a', 'd', 'b'] 
set_stopWords = stopwords.words("russian") + words
set_stopWords = set(set_stopWords)
stop = ['др. греч.', 'греч.', 'лат.', 'др.', 'и т. д', 'и т.д', 'т. е.', 'т.е.', 'т.н.', 'т. н.', 'т.п.', 'т. п.',
        'т. д', 'т.д', 'мм. рт. ст.', 'мм рт. ст.', 'мм.рт.ст', 'МЕДПОИСК.РУ', 'Контактная форма ниже', 'forbes.kz']


In [4]:
s = requests.Session()
bolezni = pd.read_csv('bolezni.csv')
# bolezni['name']

In [9]:
url = 'http://www.prostodoctor.com.ua/spravochniki/bolezni'
urls = {}
page = s.get(url)
soup = BeautifulSoup(page.content, 'lxml')
mu = soup.find('div', {'class': 'alphabet'})
mu.find('div', {'class': 'alphabet__box'}).decompose()
mu = mu.find_all('li')
for i in mu:
    urls[i.text]= 'http://www.prostodoctor.com.ua'+i.find('a')['href']
len(urls)

624

In [18]:
texts = []
num=0
for i in urls:
    page = s.get(urls[i])
    soup = BeautifulSoup(page.content, 'lxml')
    try:
        mu = soup.find('div', {'class': 'tariff__box'})
        text = mu.find('p').text
    except:
        texts.append("")
        continue
    for j in stop:
        text = text.replace(j, '')
        text = text.replace('?', '.')
        text = re.sub('\d+', ' ', text)
        text = re.sub('[^\w.]', ' ', text)

        final_s = ''
        sentences = text.split('.')
        for sen in sentences:
            sen = sen.strip()
            for tok in sen.split(' '):
                tok = tok.strip()
                if(tok.lower() not in set_stopWords):
                    final_s += (tok + ' ')
            final_s += '.'

    texts.append(final_s)


In [13]:
len(texts)

218

In [ ]:
uri = []
name =[]
for i in urls:
    uri.append(urls[i])
    name.append(i)

In [ ]:
res = pd.DataFrame()
res['name'] = name
res['url'] = uri
res['descr'] = texts
res.head()
res.to_csv('bolezni_prostodoctor.csv', index=False)

In [ ]:
a = set()
num = 0
for i in bolezni['name']:
    if i in urls:
          num += 1